[![Colabで開く](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/schwalbe1996/ds_media_intro/blob/main/chap14.ipynb)

# 14章 「画像からの特徴抽出」

In [ ]:
!wget -nc https://github.com/schwalbe1996/ds_media_intro/raw/main/sample.png
!wget -nc https://github.com/schwalbe1996/ds_media_intro/raw/main/roadscene.png
!wget -nc https://github.com/schwalbe1996/ds_media_intro/raw/main/stopsign.png

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# ノイズに対する影響を確認するため，ガウシアンノイズを加える
image = cv2.imread('sample.png',cv2.IMREAD_GRAYSCALE)
sigma = 5
noise = np.random.normal(0, sigma, image.shape)
image= image + noise
    
# 単にラプラシアンフィルタを適用したもの
K_laplacian = np.array([[0,1,0],[1,-4,1],[0,1,0]])
result_l = cv2.filter2D(image, ddepth=cv2.CV_64F, kernel=K_laplacian)

# DoGフィルタを適用したもの
sigma1, sigma2 = 1.3, 2.6
ret1 = cv2.GaussianBlur(image, (3,3), sigma1)
ret2 = cv2.GaussianBlur(image, (3,3), sigma2)
dog = ret1 - ret2

# Canny法
edges = cv2.Canny(image.astype(np.uint8), 100, 200)

plt.figure(figsize=(12,8))
plt.subplot(2,2,1)
plt.imshow(image, cmap='gray')
plt.subplot(2,2,2)
plt.imshow(np.abs(result_l), cmap='gray')
plt.subplot(2,2,3)
plt.imshow(np.abs(dog),cmap='gray')
plt.subplot(2,2,4)
plt.imshow(edges, cmap='gray')
plt.show()

In [ ]:
# 変数edgeはCanny法でエッジ検出した結果
lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=50, minLineLength=20, maxLineGap=3)
print(f'{lines.shape[0]}本の線分が検出されました')
    
disp = image.copy()
for l in lines: # 検出された各線分ごとに繰り返し
    x0, y0, x1, y1 = l[0]
    # 点(x0,y0)と点(x1,y1)とを結ぶ線分を画像に書き込む
    cv2.line(disp, (x0,y0), (x1,y1), color=(255), thickness=2)    
    
plt.imshow(disp,cmap='gray')
plt.show()

In [ ]:
image = cv2.imread('sample.png', cv2.IMREAD_GRAYSCALE)
SIFT = cv2.SIFT_create()
keypoints, descriptors = SIFT.detectAndCompute(image, None)
print(f'{len(keypoints)}個のキーポイントが検出')
print(f'ひとつ目のキーポイントの位置と向き：{keypoints[0].pt},{keypoints[0].angle}')
# print(descriptors[0]) # ひとつ目のキーポイントの特徴量（128次元ベクトル）
disp = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.imshow(disp)
plt.show()

In [ ]:
image = cv2.imread('roadscene.png', cv2.IMREAD_GRAYSCALE)
sign = cv2.imread('stopsign.png' , cv2.IMREAD_GRAYSCALE)
    
SIFT = cv2.SIFT_create()
keypoints, descriptors = SIFT.detectAndCompute(image, None)
s_keypoints, s_descriptors = SIFT.detectAndCompute(sign, None)
disp = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
s_disp = cv2.drawKeypoints(sign, s_keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.imshow(disp)
plt.show()
plt.imshow(s_disp)
plt.show()

In [ ]:
flann = cv2.FlannBasedMatcher(dict(algorithm=1, trees=5), dict(checks=50))
matches = flann.knnMatch(s_descriptors, descriptors, k=2)   
good = []
for best,second in matches:
    if best.distance < 0.7*second.distance:
        good.append([best])
print(f'{len(good)}個の対応点が見つかりました')
for m in good:
    s_pt = s_keypoints[m[0].queryIdx].pt
    pt = keypoints[m[0].trainIdx].pt
    print(f'標識:{s_pt}⇒画像:{pt}')

In [ ]:
disp = cv2.drawMatchesKnn(sign,s_keypoints,image,keypoints,good,None)
plt.imshow(disp)
plt.show()